### GAN Attempt!

In [1]:
!pip install transformers
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 7.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 41.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 13.2 MB/s eta 0:00:0

In [2]:
"""
Setup
"""

import torch
import torch.nn as nn
import torch.optim as optim
from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    AutoModel,
)
import pandas as pd
from datasets import Dataset

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

%cd drive/MyDrive/CS\ 224N/CS\ 224N\ Project
%ls # verify that you are in the right directory

Mounted at /content/drive
/content/drive/.shortcut-targets-by-id/1x7bmzM_qtbj3CPKzenuwvwocK9KiZzWU/CS 224N Project
'224N Experiments (GAN on 998 Samples).gsheet'
'224N Project Brainstorm.gdoc'
'224N Project Helpful Tutorials.gdoc'@
'224N Project Milestone Notes.gdoc'
 Adversarial-T5_Structure_1.ipynb
 aita_clean.csv
 aita_comments.csv
 aita_test_set.csv
 aita_test_set.gsheet
 aita_train_set.csv
 aita_valid_set.csv
 banana/
 banana.ipynb
 bart-base-checkpoint-204000/
 bart-baseline-attempt2.ipynb
 bart-checkpoint-5000/
 bert-baseline.ipynb
 blueberry/
 blueberry.ipynb
 checkpoint.txt
 config.json
 csvs/
 dataset_agg.ipynb
 drive/
 Evaluate.ipynb
'experimenting with gumbel.ipynb'
 finetune-gpt2/
'first proposal OLD'/
 gan-gen-trial
 gan-halfway-gen-transformer-d
 gpt-2-attempt2.ipynb
 gpt2-attempt3.ipynb
 gpt2-baseline.ipynb
 gpt2-small-rationale-generation/
 gpt2-wt-5.ipynb
 grape/
 grape-exploration.ipynb
 grape.ipynb
 honeydew/
 honeydew.ipynb
 logs/
 Mango.ipynb
 mango-old/
 orange-h

In [6]:
"""
Define the generator (use the pre-trained BART implementation)
"""

# bart-base checkpoint pre-trained on our dataset
# (can also try generically pre-trained bart base)
model_dir = 'bart-base-checkpoint-204000'
tokenizer = AutoTokenizer.from_pretrained(model_dir)
netG = AutoModelForSeq2SeqLM.from_pretrained(model_dir)
print(netG)

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(50265, 768, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(50265, 768, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0): BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), eps=1e-05,

In [8]:
from transformers import pipeline
sentiment = pipeline(model='finiteautomata/bertweet-base-sentiment-analysis')
#sentiment = AutoModel.from_pretrained('finiteautomata/bertweet-base-sentiment-analysis')

emoji is not installed, thus not converting emoticons or emojis into text. Install emoji: pip3 install emoji==0.6.0


In [9]:
"""
Define transformer discriminator
"""

nc = 1
ndf = 64

# class Discriminator(nn.Module):
#     def __init__(self, ngpu):
#         super(Discriminator, self).__init__()
#         self.ngpu = ngpu
#         self.main = nn.Sequential(
#             #Reshaping input
#             nn.Upsample(size=(64, 64)), #bring image from 1, 1, 1, 64 --> 1, 1, 64, 64
#             # input is (nc) x 64 x 64 | our input is 1 x 64
#             nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
#             nn.LeakyReLU(0.2, inplace=True),
#             # state size. (ndf) x 32 x 32
#             nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
#             nn.BatchNorm2d(ndf * 2),
#             nn.LeakyReLU(0.2, inplace=True),
#             # state size. (ndf*2) x 16 x 16
#             nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
#             nn.BatchNorm2d(ndf * 4),
#             nn.LeakyReLU(0.2, inplace=True),
#             # state size. (ndf*4) x 8 x 8
#             nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
#             nn.BatchNorm2d(ndf * 8),
#             nn.LeakyReLU(0.2, inplace=True),
#             # state size. (ndf*8) x 4 x 4
#             nn.Conv2d(ndf * 8, 1, 4, 1, 0, bias=False),
#             nn.Sigmoid()
#         )

#     def forward(self, input):
#         #print(len(input.shape))
#         return self.main(input)

class Discriminator(nn.Module):
    def __init__(self, ngpu):
        super(Discriminator, self).__init__()
        self.ngpu = ngpu
        
        # Transformer Encoder
        self.upsample = nn.Upsample(size=(64))
        encoder_layer = nn.TransformerEncoderLayer(d_model=64, nhead=8)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=6)
        
        self.classifier = nn.Sequential(
            nn.Linear(64, 1),
            nn.Sigmoid()
        )

    def forward(self, input):
        #input shape: (batch_size, seq_len, d_model)
        upsampled_input = self.upsample(input)
        transformer_output = self.transformer_encoder(upsampled_input) 
        discriminator_output = self.classifier(transformer_output.mean(dim=1)) #(batch_size, 1)
        
        return discriminator_output


In [10]:
ngpu = 1
netD = Discriminator(ngpu).to(device)

# d_model_dir = "siebert/sentiment-roberta-large-english"
# netD = AutoModelForSequenceClassification.from_pretrained(model_dir)

#netD =   #AutoModel.from_pretrained("vinai/bertweet-base")

In [11]:
"""
Loss functions and optimizers
"""
# Size of generator input
nz = 512
# Optim params
lr = 0.0002
beta1 = 0.5

# Initialize BCELoss function
criterion = nn.BCELoss()

# Create batch of latent vectors that we will use to visualize
#  the progression of the generator
fixed_noise = torch.randn(64, nz, 1, 1, device=device)

# Establish convention for real and fake labels during training
real_label = 1.
fake_label = 0.

# Setup Adam optimizers for both G and D
optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(beta1, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(beta1, 0.999))

In [12]:
train_df = pd.read_csv('aita_train_set.csv')[['text', 'comments']]
valid_df = pd.read_csv('aita_valid_set.csv')[['text', 'comments']]
test_df = pd.read_csv('aita_test_set.csv')[['text', 'comments']]

In [13]:
valid_df

,text,comments
0,AITA for insisting on a refund?. I'm genuinely...,YTA. Why all this needless drama and attention...
1,AITA for telling her that her boyfriend creeps...,"YTA. You tried to kiss her, she wasn't interes..."
2,AITA for making my son do an exercise routine ...,Why don't you do things with him? Instead of i...
3,AITA for disagreeing with a friend that making...,"It wasn't really an ""announcement,"" everybody ..."
4,WIBTA if I message an employer about why I did...,YTA \n\nDO NOT DO THIS. Do not cross the strea...
...,...,...
993,WIBTA if I expose antivaxxers?. Back story: my...,NTA - I would say you're an asshole if you did...
994,WIBTA if I took my mini-fridge from my roommat...,"NTA, but this is largely your fault too. Who c..."
995,"WIBTA if i ""snitch"" on my half sister to my pa...",NTA whatever her motivation for crying to her ...
996,AITA for not wanting to live with my roommate ...,NTA. The lease was up; it wasn’t as if you wer...


In [14]:
train_data_txt = Dataset.from_pandas(train_df)
validation_data_txt = Dataset.from_pandas(valid_df)
test_data_txt = Dataset.from_pandas(test_df)
print(train_data_txt)
print(validation_data_txt)
print(test_data_txt)

Dataset({
    features: ['text', 'comments'],
    num_rows: 81614
})
Dataset({
    features: ['text', 'comments'],
    num_rows: 998
})
Dataset({
    features: ['text', 'comments'],
    num_rows: 998
})


In [15]:
"""
Preprocess
"""

encoder_max_length = 256  # changed from 256
decoder_max_length = 64  # changed from 64

def batch_tokenize_preprocess(batch, tokenizer, max_source_length, max_target_length):
    source, target = batch["text"], batch["comments"]
    source_tokenized = tokenizer(
        source, padding="max_length", truncation=True, max_length=max_source_length, return_tensors="pt"
    )
    target_tokenized = tokenizer(
        target, padding="max_length", truncation=True, max_length=max_target_length, return_tensors="pt"
    )

    batch = {k: v for k, v in source_tokenized.items()}
    # Ignore padding in the loss
    batch["labels"] = [
        [-100 if token == tokenizer.pad_token_id else token for token in l]
        for l in target_tokenized["input_ids"]
    ]
    return batch


train_data = train_data_txt.map(
    lambda batch: batch_tokenize_preprocess(
        batch, tokenizer, encoder_max_length, decoder_max_length
    ),
    batched=True,
    remove_columns=train_data_txt.column_names,
)

validation_data = validation_data_txt.map(
    lambda batch: batch_tokenize_preprocess(
        batch, tokenizer, encoder_max_length, decoder_max_length
    ),
    batched=True,
    remove_columns=validation_data_txt.column_names,
)

Map:   0%|          | 0/81614 [00:00<?, ? examples/s]

Map:   0%|          | 0/998 [00:00<?, ? examples/s]

In [16]:
print(train_data)

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 81614
})


In [17]:
print(train_data[0]['input_ids'])

[0, 250, 2068, 250, 13, 45, 2405, 964, 19, 5, 1816, 54, 12961, 162, 116, 4, 28106, 4384, 6, 142, 127, 1441, 333, 70, 216, 127, 44014, 1316, 36, 9226, 40, 146, 1472, 423, 322, 1437, 50118, 50118, 100, 348, 57, 4927, 10, 1816, 38, 348, 57, 964, 19, 187, 127, 3812, 76, 9, 239, 334, 4, 166, 348, 4925, 5, 276, 333, 9, 7628, 964, 187, 172, 36, 1694, 32, 70, 7195, 11, 1564, 122, 322, 166, 214, 70, 1256, 3229, 29716, 4, 50118, 50118, 10462, 6, 79, 12961, 162, 682, 4319, 69, 2188, 25, 145, 274, 3765, 673, 36, 506, 4352, 9, 1716, 66, 43, 77, 24, 606, 7, 5, 4927, 1310, 6, 941, 187, 52, 32, 70, 98, 664, 4, 1437, 50118, 50118, 100, 399, 75, 269, 7684, 69, 13, 24, 6, 142, 14, 18, 10, 8134, 1219, 8, 38, 399, 75, 206, 1169, 9, 201, 58, 27744, 10, 1473, 499, 561, 6, 23, 513, 45, 648, 4, 125, 38, 21, 6, 8, 202, 524, 6, 5278, 15418, 13, 42, 1816, 8, 38, 802, 24, 74, 28, 275, 114, 52, 2442, 45, 964, 4, 1437, 50118, 50118, 1779, 38, 174, 69, 42, 6, 79, 300, 10, 410, 828, 2465, 4, 767, 7, 69, 6, 38, 21, 145

In [18]:
print(len(train_data[0]['input_ids']))

256


In [19]:
def get_sentiment_tokens(comment):
  max_input_length = 128
  if len(comment) > 128:
    comment = comment[:128]
  comment_sentiment = sentiment([comment])[0]
  comment_sentiment = comment_sentiment['label'] + ': ' + str(comment_sentiment['score'])
  comment_sentiment_tokens = tokenizer(comment_sentiment, max_length=128, padding='max_length', truncation=True, return_tensors="pt")
  return comment_sentiment_tokens['input_ids'].tolist()

In [20]:
fixed_validation_index = 17
fixed_validation_inputs = valid_df.iloc[fixed_validation_index]['text']
fixed_validation_data = tokenizer(fixed_validation_inputs, max_length=512, padding='max_length', truncation=True, return_tensors="pt")

In [21]:
batch_size=4
dataloader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,
                                         shuffle=True)
for i, data in enumerate(dataloader, 0):
  print(torch.stack(data['attention_mask']).shape)
  break


torch.Size([256, 4])


In [22]:
# # Training Loop

# # Lists to keep track of progress
# img_list = []
# G_losses = []
# D_losses = []
# iters = 0
# # num_epochs = 1
# max_input_length = 512

# print("Starting Training Loop...")
# # For each epoch
# for epoch in range(num_epochs):
#     # todo: batch this/use a dataloader
#     for i, data in enumerate(dataloader, 0):
#         #data = train_data[i]
#         ############################
#         # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
#         ###########################
#         ## Train with all-real batch
#         netD.zero_grad()
#         # Format batch
#         #print(len(data['labels']))
#         #print(data)
#         print(data['labels'])
#         real_cpu = torch.stack(data['labels'])
#         #real_cpu = torch.unsqueeze(real_cpu, dim=0)
#         #real_cpu = torch.cat(real_cpu, dim=0)
#         #real_cpu = torch.tensor(data['labels'], dtype=torch.float32)
#         #real_cpu = data['labels']
#         #print(real_cpu.shape)
#         # real_cpu = real_cpu.unsqueeze(0)
#         # real_cpu = real_cpu.unsqueeze(0)
#         # real_cpu = real_cpu.unsqueeze(0)
#         real_cpu = real_cpu.type(torch.FloatTensor).view(-1, 1, 1, 64) #these are the comment tokens
#         #print(real_cpu.shape)
#         real_cpu = real_cpu.to(device)
#         b_size = real_cpu.size(0)
#         label = torch.full((b_size,), real_label, dtype=torch.float, device=device)

#         #discriminator will train off of true comments in the real batch pass
#         # Forward pass real batch through D
#         output = netD(real_cpu).view(-1) 
#         # Calculate loss on all-real batch
#         errD_real = criterion(output, label)
#         # Calculate gradients for D in backward pass
#         errD_real.backward()
#         D_x = output.mean().item()

#         ## Train with all-fake batch
#         # Generate batch of latent vectors
#         # noise = torch.randn(b_size, nz, 1, 1, device=device)
#         # print(inputs['input_ids'].shape)
#         # output = model.generate(**inputs, num_beams=8, do_sample=True, min_length=10, max_length=64)
#         # Generate fake image batch with G

#         #inputs = train_df.iloc[i]['text']
#         #data = tokenizer(inputs, max_length=max_input_length, padding='max_length', truncation=True, return_tensors="pt")
#         data = {k : torch.stack(v) for k, v in data.items()}
#         fake = netG(**data)

#         # fake = netG.generate(**data, num_beams=8, do_sample=True, min_length=10, max_length=64) #generate a fake comment
#         label.fill_(fake_label)
#         # Classify all fake batch with D
#         #print(fake.shape)
#         fake = fake.type(torch.float32)
#         fake = fake.view(1, 1, 1, -1)
#         fake = fake.detach().to(device)
#         output = netD(fake).view(-1)
#         # Calculate D's loss on the all-fake batch
#         errD_fake = criterion(output, label)
#         # Calculate the gradients for this batch, accumulated (summed) with previous gradients
#         errD_fake.backward()
#         D_G_z1 = output.mean().item()
#         # Compute error of D as sum over the fake and the real batches
#         errD = errD_real + errD_fake
#         # Update D
#         optimizerD.step()

#         ############################
#         # (2) Update G network: maximize log(D(G(z)))
#         ###########################
#         netG.zero_grad()
#         label.fill_(real_label)  # fake labels are real for generator cost
#         # Since we just updated D, perform another forward pass of all-fake batch through D
#         output = netD(fake).view(-1)
#         # Calculate G's loss based on this output
#         errG = criterion(output, label)
#         # Calculate gradients for G
#         errG.backward()
#         D_G_z2 = output.mean().item()
#         # Update G
#         optimizerG.step()

#         # Output training stats
#         if i % 5 == 0:
#             print('[%d/%d][%d/%d]\tLoss_D: %.4f\tLoss_G: %.4f\tD(x): %.4f\tD(G(z)): %.4f / %.4f'
#                   % (epoch, num_epochs, i, len(train_data),
#                      errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))

#         # Save Losses for plotting later
#         G_losses.append(errG.item())
#         D_losses.append(errD.item())

#         # Check how the generator is doing by saving G's output on fixed_noise
#         if (iters % 500 == 0) or ((epoch == num_epochs-1) and (i == len(train_data)-1)):
#             with torch.no_grad():
#                 fake = netG.generate(**fixed_validation_data, num_beams=8, do_sample=True, min_length=10, max_length=64).detach()
#             # img_list.append(vutils.make_grid(fake, padding=2, normalize=True))

#         iters += 1

In [23]:
################ Changing the working data to validation ####################

# Training Loop

# Lists to keep track of progress
img_list = []
G_losses = []
D_losses = []
iters = 0
num_epochs = 1
max_input_length = 512

print("Starting Training Loop...")
# For each epoch
for epoch in range(num_epochs):
    # todo: batch this/use a dataloader
    for i in range(len(validation_data)):
        data = validation_data[i]
        ############################
        # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
        ###########################
        ## Train with all-real batch
        netD.zero_grad()
        # Format batch
        #print(len(data['labels']))

        sentiment_tokens = get_sentiment_tokens(str(valid_df.iloc[i]['comments']))
        #print(data['labels'])
        #print(sentiment_tokens[0])
        real_cpu = torch.tensor(data['labels'] + sentiment_tokens[0], dtype=torch.float32)
        #real_cpu = data['labels']
        #print(real_cpu.shape)
        # real_cpu = real_cpu.unsqueeze(0)
        # real_cpu = real_cpu.unsqueeze(0)
        # real_cpu = real_cpu.unsqueeze(0)
        #print(real_cpu.shape)
        real_cpu = real_cpu.view(1, 1, 64 + 128) #these are the comment tokens
        #print(real_cpu.shape)
        #print(real_cpu.shape)
        real_cpu = real_cpu.to(device)
        b_size = real_cpu.size(0)
        label = torch.full((b_size,), real_label, dtype=torch.float, device=device)

        #discriminator will train off of true comments in the real batch pass
        # Forward pass real batch through D
        output = netD(real_cpu).view(-1) 
        # Calculate loss on all-real batch
        errD_real = criterion(output, label)
        # Calculate gradients for D in backward pass
        errD_real.backward()
        D_x = output.mean().item()

        ## Train with all-fake batch
        # Generate batch of latent vectors
        # noise = torch.randn(b_size, nz, 1, 1, device=device)
        # print(inputs['input_ids'].shape)
        # output = model.generate(**inputs, num_beams=8, do_sample=True, min_length=10, max_length=64)
        # Generate fake image batch with G

        inputs = valid_df.iloc[i]['text']
        data = tokenizer(inputs, max_length=max_input_length, padding='max_length', truncation=True, return_tensors="pt")
        fake = netG.generate(**data, num_beams=8, do_sample=True, min_length=10, max_length=64) #generate a fake comment
        decoded_fake_comment = tokenizer.batch_decode(fake, skip_special_tokens=True)
        sentiment_tokens = get_sentiment_tokens(decoded_fake_comment[0])
        #print(decoded_fake_comment[0])
        #print(type(decoded_fake_comment[0]))
        #print(type(sentiment(decoded_fake_comment[0])[0]))
        #print(sentiment_tokens[0])
        inp_tensor = torch.tensor(sentiment_tokens[0], dtype=torch.long).unsqueeze(0)
        fake = torch.cat((inp_tensor, fake), dim=1)
        label.fill_(fake_label)
        # Classify all fake batch with D
        #print(fake.shape)
        fake = fake.type(torch.float32)
        fake = fake.view(1, 1, -1)
        #print(fake.shape)
        #fake = correct_to_64(fake)
        #print(fake.shape)
        fake = fake.detach().to(device)
        output = netD(fake).view(-1)
        # Calculate D's loss on the all-fake batch
        errD_fake = criterion(output, label)
        # Calculate the gradients for this batch, accumulated (summed) with previous gradients
        errD_fake.backward()
        D_G_z1 = output.mean().item()
        # Compute error of D as sum over the fake and the real batches
        errD = errD_real + errD_fake
        # Update D
        optimizerD.step()

        ############################
        # (2) Update G network: maximize log(D(G(z)))
        ###########################
        netG.zero_grad()
        label.fill_(real_label)  # fake labels are real for generator cost
        # Since we just updated D, perform another forward pass of all-fake batch through D
        output = netD(fake).view(-1)
        # Calculate G's loss based on this output
        errG = criterion(output, label)
        # Calculate gradients for G
        errG.backward()
        D_G_z2 = output.mean().item()
        # Update G
        optimizerG.step()

        # Output training stats
        if i % 5 == 0:
            print('[%d/%d][%d/%d]\tLoss_D: %.4f\tLoss_G: %.4f\tD(x): %.4f\tD(G(z)): %.4f / %.4f'
                  % (epoch, num_epochs, i, len(validation_data),
                     errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))

        # Save Losses for plotting later
        G_losses.append(errG.item())
        D_losses.append(errD.item())

        if iters == 5: netG.save_pretrained('orange/orange-trial')
        # Check how the generator is doing by saving G's output on fixed_noise
        if (iters % 500 == 0) or ((epoch == num_epochs-1) and (i == len(train_data)-1)):
            netG.save_pretrained('orange/orange-halfway')
            with torch.no_grad():
                fake = netG.generate(**fixed_validation_data, num_beams=8, do_sample=True, min_length=10, max_length=64).detach()
            # img_list.append(vutils.make_grid(fake, padding=2, normalize=True))

        iters += 1

netG.save_pretrained('orange/orange-full')

Starting Training Loop...
[0/1][0/998]	Loss_D: 1.1869	Loss_G: 0.8342	D(x): 0.8622	D(G(z)): 0.6461 / 0.4342
[0/1][5/998]	Loss_D: 1.8239	Loss_G: 1.5843	D(x): 0.2806	D(G(z)): 0.4248 / 0.2051
[0/1][10/998]	Loss_D: 0.6056	Loss_G: 1.3363	D(x): 0.8479	D(G(z)): 0.3563 / 0.2628
[0/1][15/998]	Loss_D: 0.1448	Loss_G: 2.5036	D(x): 0.9524	D(G(z)): 0.0916 / 0.0818
[0/1][20/998]	Loss_D: 0.1120	Loss_G: 2.8985	D(x): 0.9676	D(G(z)): 0.0761 / 0.0551
[0/1][25/998]	Loss_D: 0.1253	Loss_G: 2.6990	D(x): 0.9618	D(G(z)): 0.0827 / 0.0673
[0/1][30/998]	Loss_D: 0.1049	Loss_G: 3.0962	D(x): 0.9649	D(G(z)): 0.0668 / 0.0452
[0/1][35/998]	Loss_D: 3.2399	Loss_G: 2.5500	D(x): 0.0406	D(G(z)): 0.0343 / 0.0781
[0/1][40/998]	Loss_D: 0.1362	Loss_G: 3.1790	D(x): 0.9144	D(G(z)): 0.0457 / 0.0416
[0/1][45/998]	Loss_D: 0.0602	Loss_G: 3.2891	D(x): 0.9763	D(G(z)): 0.0356 / 0.0373
[0/1][50/998]	Loss_D: 0.2191	Loss_G: 1.0043	D(x): 0.9749	D(G(z)): 0.1761 / 0.3663
[0/1][55/998]	Loss_D: 2.6487	Loss_G: 2.1784	D(x): 0.0755	D(G(z)): 0.0636 /

In [ ]:
# ################ WORKING DO NOT TOUCH ####################

# # Training Loop

# # Lists to keep track of progress
# img_list = []
# G_losses = []
# D_losses = []
# iters = 0
# num_epochs = 1
# max_input_length = 512

# print("Starting Training Loop...")
# # For each epoch
# for epoch in range(num_epochs):
#     # todo: batch this/use a dataloader
#     for i in range(len(train_data)):
#         data = train_data[i]
#         ############################
#         # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
#         ###########################
#         ## Train with all-real batch
#         netD.zero_grad()
#         # Format batch
#         #print(len(data['labels']))
#         real_cpu = torch.tensor(data['labels'], dtype=torch.float32)
#         #real_cpu = data['labels']
#         #print(real_cpu.shape)
#         # real_cpu = real_cpu.unsqueeze(0)
#         # real_cpu = real_cpu.unsqueeze(0)
#         # real_cpu = real_cpu.unsqueeze(0)
#         real_cpu = real_cpu.view(1, 1, 1, 64) #these are the comment tokens
#         #print(real_cpu.shape)
#         real_cpu = real_cpu.to(device)
#         b_size = real_cpu.size(0)
#         label = torch.full((b_size,), real_label, dtype=torch.float, device=device)

#         #discriminator will train off of true comments in the real batch pass
#         # Forward pass real batch through D
#         output = netD(real_cpu).view(-1) 
#         # Calculate loss on all-real batch
#         errD_real = criterion(output, label)
#         # Calculate gradients for D in backward pass
#         errD_real.backward()
#         D_x = output.mean().item()

#         ## Train with all-fake batch
#         # Generate batch of latent vectors
#         # noise = torch.randn(b_size, nz, 1, 1, device=device)
#         # print(inputs['input_ids'].shape)
#         # output = model.generate(**inputs, num_beams=8, do_sample=True, min_length=10, max_length=64)
#         # Generate fake image batch with G

#         inputs = train_df.iloc[i]['text']
#         data = tokenizer(inputs, max_length=max_input_length, padding='max_length', truncation=True, return_tensors="pt")
#         fake = netG.generate(**data, num_beams=8, do_sample=True, min_length=10, max_length=64) #generate a fake comment
#         label.fill_(fake_label)
#         # Classify all fake batch with D
#         #print(fake.shape)
#         fake = fake.type(torch.float32)
#         fake = fake.view(1, 1, 1, -1)
#         fake = fake.detach().to(device)
#         output = netD(fake).view(-1)
#         # Calculate D's loss on the all-fake batch
#         errD_fake = criterion(output, label)
#         # Calculate the gradients for this batch, accumulated (summed) with previous gradients
#         errD_fake.backward()
#         D_G_z1 = output.mean().item()
#         # Compute error of D as sum over the fake and the real batches
#         errD = errD_real + errD_fake
#         # Update D
#         optimizerD.step()

#         ############################
#         # (2) Update G network: maximize log(D(G(z)))
#         ###########################
#         netG.zero_grad()
#         label.fill_(real_label)  # fake labels are real for generator cost
#         # Since we just updated D, perform another forward pass of all-fake batch through D
#         output = netD(fake).view(-1)
#         # Calculate G's loss based on this output
#         errG = criterion(output, label)
#         # Calculate gradients for G
#         errG.backward()
#         D_G_z2 = output.mean().item()
#         # Update G
#         optimizerG.step()

#         # Output training stats
#         if i % 5 == 0:
#             print('[%d/%d][%d/%d]\tLoss_D: %.4f\tLoss_G: %.4f\tD(x): %.4f\tD(G(z)): %.4f / %.4f'
#                   % (epoch, num_epochs, i, len(train_data),
#                      errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))

#         # Save Losses for plotting later
#         G_losses.append(errG.item())
#         D_losses.append(errD.item())

#         # Check how the generator is doing by saving G's output on fixed_noise
#         if (iters % 500 == 0) or ((epoch == num_epochs-1) and (i == len(train_data)-1)):
#             with torch.no_grad():
#                 fake = netG.generate(**fixed_validation_data, num_beams=8, do_sample=True, min_length=10, max_length=64).detach()
#             # img_list.append(vutils.make_grid(fake, padding=2, normalize=True))

#         iters += 1

In [ ]:
# import torch
# import torch.nn as nn

# input = torch.randn(1, 64).view(1, 1, 1, 64)
# print(input.shape)
# m = nn.Upsample(size=(64, 64))
# output = m(input)
# output = output.reshape((1, 64, 64))
# print(output.shape)

# up = nn.Upsample(size=(24, 24))

# x = torch.randn(1, 3, 10, 10)
# print(up(x).shape)

In [ ]:
# up = nn.Upsample(size=(24, 24))

# x = torch.randn(1, 3, 10, 10)
# print(up(x).shape)

In [ ]:
# input = torch.randn(1, 64).view(1, 1, 1, 64)
# m = nn.Upsample(size=(64, 64))
# intermediate = m(input)
# x = nn.Flatten(0, 1)
# output = x(intermediate)
# print(input.shape)
# print(intermediate.shape)
# print(output.shape)